In [6]:
sergio_dir = "/groups/itay_mayrose/halabikeren/programs/SERGIO/"

In [7]:
import numpy as np
import pandas as pd
import networkx as nx
import scipy
import os

import sys
sys.path.append(sergio_dir)
from SERGIO.sergio import sergio

In [8]:
def get_expr_with_noise(expr, sim, dropout_percentile):
    """
    Add outlier genes
    """
    expr_O = sim.outlier_effect(expr, outlier_prob = 0.01, mean = 0.8, scale = 1)

    """
    Add Library Size Effect
    """
    libFactor, expr_O_L = sim.lib_size_effect(expr_O, mean = 4.6, scale = 0.4)

    """
    Add Dropouts
    """
    binary_ind = sim.dropout_indicator(expr_O_L, shape = 6.5, percentile = dropout_percentile)
    expr_O_L_D = np.multiply(binary_ind, expr_O_L)

    """
    Convert to UMI count
    """
    count_matrix_wo_dropouts = sim.convert_to_UMIcounts(expr_O_L)
    count_matrix_w_dropouts = sim.convert_to_UMIcounts(expr_O_L_D)

    """
    Make a 2d gene expression matrix
    """
    count_matrix_wo_dropouts = np.concatenate(count_matrix_wo_dropouts, axis = 1)
    count_matrix_w_dropouts = np.concatenate(count_matrix_w_dropouts, axis = 1)
       
    return count_matrix_wo_dropouts, count_matrix_w_dropouts

In [9]:
ngenes = 100
ncell_types = 3
ds_name = "De-noised_400G_9T_300cPerT_5_DS2"
input_file_taregts = f'{sergio_dir}data_sets/{ds_name}/Interaction_cID_5.txt'
input_file_regs=f'{sergio_dir}data_sets/{ds_name}/Regs_cID_5.txt'
if ngenes == 100:
    if ncell_types == 9:
        ds_name = "De-noised_100G_9T_300cPerT_4_DS1"
        input_file_taregts = f'{sergio_dir}data_sets/{ds_name}/Interaction_cID_4.txt' 
        input_file_regs=f'{sergio_dir}data_sets/{ds_name}/Regs_cID_4.txt'
    elif ncell_types == 3:
        ds_name = "De-noised_100G_3T_300cPerT_dynamics_8_DS8"
        input_file_taregts = f'{sergio_dir}data_sets/{ds_name}/Interaction_cID_8.txt' 
        input_file_regs=f'{sergio_dir}data_sets/{ds_name}/Regs_cID_8.txt'

n_cells = 50
sampling_state = 15
noise_params_options = [1,1.5,2]
noise_type_options = ["dpd", "sp", "dpd"]
decays_options = [0.6, 0.8, 1]
dropout_percentiles = [20, 40, 60]

output_dir = f"../data/simulations/{ds_name}_{len(noise_params_options)}_tech_partitions/"
os.makedirs(output_dir, exist_ok=True)

In [10]:
sim1 = sergio(number_genes=ngenes, number_bins = ncell_types, number_sc = n_cells, noise_params = noise_params_options[0], decays=decays_options[0], sampling_state=sampling_state, noise_type=noise_type_options[0])
sim1.build_graph(input_file_taregts =input_file_taregts, input_file_regs=input_file_regs, shared_coop_state=2)
sim1.simulate()
expr1 = sim1.getExpressions() #This returns a 3d numpy array (#cell_types * #genes * #cells_per_type)
count_matrix1_wo_dropouts, count_matrix1_w_dropouts = get_expr_with_noise(expr=expr1,
                    sim=sim1,
                    dropout_percentile=dropout_percentiles[0])
cell_type_labels1 = pd.Series(np.array([[f"cell_{i}"]*n_cells for i in range(ncell_types)]).flatten())
technology_labels1 = pd.Series(np.repeat(f"tech_1", len(cell_type_labels1)))
count_matrix1_wo_dropouts = pd.DataFrame(count_matrix1_wo_dropouts, 
             index=[f"gene_{i}" for i in range(count_matrix1_wo_dropouts.shape[0])],
             columns=[f"cell_t1_{i}" for i in range(count_matrix1_wo_dropouts.shape[1])])
count_matrix1_w_dropouts = pd.DataFrame(count_matrix1_w_dropouts, 
             index=[f"gene_{i}" for i in range(count_matrix1_w_dropouts.shape[0])],
             columns=[f"cell_t1_{i}" for i in range(count_matrix1_w_dropouts.shape[1])])

Start simulating new level
There are 3 genes to simulate in this layer
Done with current level
Start simulating new level
There are 7 genes to simulate in this layer
Done with current level
Start simulating new level
There are 90 genes to simulate in this layer
Done with current level


In [11]:
sim2 = sergio(number_genes=ngenes, number_bins = ncell_types, number_sc = n_cells, noise_params = noise_params_options[1], decays=decays_options[1], sampling_state=sampling_state, noise_type=noise_type_options[1])
sim2.build_graph(input_file_taregts=input_file_taregts, input_file_regs=input_file_regs, shared_coop_state=2)
sim2.simulate()
expr2 = sim2.getExpressions()
count_matrix2_wo_dropouts, count_matrix2_w_dropouts = get_expr_with_noise(expr=expr2,
                    sim=sim2,
                    dropout_percentile=dropout_percentiles[1])
cell_type_labels2 = pd.Series(np.array([[f"cell_{i}"]*n_cells for i in range(ncell_types)]).flatten())
technology_labels2 = pd.Series(np.repeat("tech_2", len(cell_type_labels2)))
count_matrix2_wo_dropouts = pd.DataFrame(count_matrix2_wo_dropouts, 
             index=[f"gene_{i}" for i in range(count_matrix2_wo_dropouts.shape[0])],
             columns=[f"cell_t2_{i}" for i in range(count_matrix2_wo_dropouts.shape[1])])
count_matrix2_w_dropouts = pd.DataFrame(count_matrix2_w_dropouts, 
             index=[f"gene_{i}" for i in range(count_matrix2_w_dropouts.shape[0])],
             columns=[f"cell_t2_{i}" for i in range(count_matrix2_w_dropouts.shape[1])])

Start simulating new level
There are 3 genes to simulate in this layer
Done with current level
Start simulating new level
There are 7 genes to simulate in this layer
Done with current level
Start simulating new level
There are 90 genes to simulate in this layer
Done with current level


In [12]:
sim3 = sergio(number_genes=ngenes, number_bins = ncell_types, number_sc = n_cells, noise_params = noise_params_options[2], decays=decays_options[2], sampling_state=sampling_state, noise_type=noise_type_options[2])
sim3.build_graph(input_file_taregts=input_file_taregts, input_file_regs=input_file_regs, shared_coop_state=2)
sim3.simulate()
expr3 = sim3.getExpressions()
count_matrix3_wo_dropouts, count_matrix3_w_dropouts = get_expr_with_noise(expr=expr3,
                    sim=sim3,
                    dropout_percentile=dropout_percentiles[2])
cell_type_labels3 = pd.Series(np.array([[f"cell_{i}"]*n_cells for i in range(ncell_types)]).flatten())
technology_labels3 = pd.Series(np.repeat("tech_3", len(cell_type_labels3)))
count_matrix3_wo_dropouts = pd.DataFrame(count_matrix3_wo_dropouts, 
             index=[f"gene_{i}" for i in range(count_matrix3_wo_dropouts.shape[0])],
             columns=[f"cell_t3_{i}" for i in range(count_matrix3_wo_dropouts.shape[1])])
count_matrix3_w_dropouts = pd.DataFrame(count_matrix3_w_dropouts, 
             index=[f"gene_{i}" for i in range(count_matrix3_w_dropouts.shape[0])],
             columns=[f"cell_t3_{i}" for i in range(count_matrix3_w_dropouts.shape[1])])

Start simulating new level
There are 3 genes to simulate in this layer
Done with current level
Start simulating new level
There are 7 genes to simulate in this layer
Done with current level
Start simulating new level
There are 90 genes to simulate in this layer
Done with current level


In [13]:
count_matrix_wo_dropouts = pd.concat([count_matrix1_wo_dropouts, 
                                      count_matrix2_wo_dropouts, 
                                      count_matrix3_wo_dropouts], axis=1)
count_matrix_wo_dropouts.index.rename("GENE_ID", inplace=True)
for c in count_matrix_wo_dropouts.columns:
    count_matrix_wo_dropouts[c] = pd.to_numeric(count_matrix_wo_dropouts[c])
count_matrix_wo_dropouts.to_csv(f"{output_dir}count_matrix_wo_dropouts.tsv", sep="\t")
print(f"# genes = {count_matrix_wo_dropouts.shape[0]:,}")
print(f"# cells = {count_matrix_wo_dropouts.shape[1]:,}")

count_matrix_w_dropouts = pd.concat([count_matrix1_w_dropouts, 
                                      count_matrix2_w_dropouts, 
                                      count_matrix3_w_dropouts], axis=1)
count_matrix_w_dropouts.index.rename("GENE_ID", inplace=True)
for c in count_matrix_w_dropouts.columns:
    count_matrix_w_dropouts[c] = pd.to_numeric(count_matrix_w_dropouts[c])
count_matrix_w_dropouts.to_csv(f"{output_dir}count_matrix_w_dropouts.tsv", sep="\t")

cell_type_labels = pd.concat([cell_type_labels1, cell_type_labels2, cell_type_labels3])                    
print(f"# cells type labels = {cell_type_labels.shape[0]:,}")
cell_type_labels.to_csv(f"{output_dir}cell_type_labels.txt", index=False, header=False)
with open(f"{output_dir}cell_type_labels.txt", "r") as f:
    c=f.read()[:-1]
with open(f"{output_dir}cell_type_labels.txt", "w") as f:
    f.write(c)

technology_labels = pd.concat([technology_labels1, technology_labels2, technology_labels3])                                                                                                
print(f"# technology labels = {technology_labels.shape[0]:,}")
technology_labels.to_csv(f"{output_dir}technology_labels.txt", index=False, header=False)
with open(f"{output_dir}technology_labels.txt", "r") as f:
    c=f.read()[:-1]
with open(f"{output_dir}technology_labels.txt", "w") as f:
    f.write(c)

# genes = 100
# cells = 450
# cells type labels = 450
# technology labels = 450


In [14]:
def report_stats(count_matrix):
    print(f"maximal dropout rate across cells = {((count_matrix == 0).sum()/ count_matrix.shape[0]).max():,}")
    print(f"minimal dropout rate across cells = {((count_matrix == 0).sum()/ count_matrix.shape[0]).min():,}")

In [15]:
report_stats(count_matrix_wo_dropouts)

maximal dropout rate across cells = 0.79
minimal dropout rate across cells = 0.2


In [16]:
report_stats(count_matrix_w_dropouts)

maximal dropout rate across cells = 0.94
minimal dropout rate across cells = 0.2
